### EXTRAÇÃO DOS DADOS 
Processo de conexão e requisição via <i>API</i>, a requisição foi dividia em dois blocos: <hr>

In [1]:
import pandas as pd
import requests

###### 1.0 - REQUISIÇÃO LISTA DE DEPUTADOS
Requisição dos deputados em mantato na API Dados Abertos

In [7]:
# REQUISIÇÃO VIA API 
url_requisicao_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados?&ordem=ASC&ordenarPor=nome"
resultado = requests.get(url_requisicao_deputados)
deputados = pd.DataFrame(resultado.json()['dados'])

# PROCESSO DE EXCLUSÃO DE DADOS DESNECESSÁRIOS
lista_exclusao_deputados = ['uri', 'uriPartido', 'urlFoto', 'idLegislatura']
i = 0

while i < len(lista_exclusao_deputados):
    del deputados[lista_exclusao_deputados[i]] 
    i += 1

# TRATAMENTO DOS DADOS
deputados.nome = [str(nome).upper() for nome in deputados.nome]
deputados.email = [str(email).upper() for email in deputados.email]

deputados.columns = ['id','nome','siglaPartido','ufLegislacao','email']

deputados.tail()

,id,nome,siglaPartido,ufLegislacao,email
508,178889,ZÉ CARLOS,PT,MA,DEP.ZECARLOS@CAMARA.LEG.BR
509,204559,ZÉ NETO,PT,BA,DEP.ZENETO@CAMARA.LEG.BR
510,160632,ZÉ SILVA,SOLIDARIEDADE,MG,DEP.ZESILVA@CAMARA.LEG.BR
511,204517,ZÉ VITOR,PL,MG,DEP.ZEVITOR@CAMARA.LEG.BR
512,160592,ZECA DIRCEU,PT,PR,DEP.ZECADIRCEU@CAMARA.LEG.BR


###### 1.1 - REQUISIÇÃO DETALHE DOS DEPUTADOS
Informações adicionais dos deputados

In [8]:
# VARIÁVEIS
id_deputado = deputados['id']
lista_id = []
lista_escolaridade = []
lista_data_nascimento = []
lista_municipio_nascimento = []
lista_sexo = []
lista_estado_nascimento = []

# FUNÇÃO QUE PEGA O ID DO DEPUTADO, FAZ REQUISIÇÃO DA API E SALVA EM UMA VARIÁVEL
def consulta_deputados():
    i = 0
    while i < len(id_deputado):
        conn = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados/"+str(id_deputado[i]))
        JSON = conn.json()['dados']
        # CONSULTA DADOS 
        lista_id.extend([JSON['id']])
        lista_data_nascimento.extend([JSON['dataNascimento']])
        lista_escolaridade.extend([JSON['escolaridade']])
        lista_municipio_nascimento.extend([JSON['municipioNascimento']])
        lista_sexo.extend([JSON['sexo']])
        lista_estado_nascimento.extend([JSON['ufNascimento']])
        i += 1 

consulta_deputados()

# PROCESSO DE SALVAR OS DADOS EM UM DATAFRAME E RENOMEAR COLUNAS
deputados_detalhe = pd.DataFrame(list(zip(lista_id,lista_escolaridade,lista_data_nascimento,lista_sexo,lista_estado_nascimento, lista_municipio_nascimento)))
deputados_detalhe.columns=['id','escolaridade','dataNascimento','sexo','estadoNascimento','cidadeNascimento']


# TRATAMENTO DOS DADOS
deputados_detalhe.escolaridade = [str(escolaridade).upper() for escolaridade in deputados_detalhe.escolaridade]
deputados_detalhe.cidadeNascimento = [str(cidadeNascimento).upper() for cidadeNascimento in deputados_detalhe.cidadeNascimento]

deputados_detalhe.head()

,id,escolaridade,dataNascimento,sexo,estadoNascimento,cidadeNascimento
0,204554,SUPERIOR INCOMPLETO,1965-02-13,M,BA,SALVADOR
1,204521,SUPERIOR,1966-11-06,M,SP,SÃO PAULO
2,204379,SUPERIOR,1983-09-28,M,AP,MACAPÁ
3,204560,SUPERIOR,1981-02-02,M,BA,SALVADOR
4,204528,DOUTORADO,1969-03-06,F,SP,SÃO PAULO


###### 1.2 - JOIN DAS TABELAS DEPUTADOS E DEPUTADOS DETALHE

In [10]:
deputados_join = pd.merge(deputados, deputados_detalhe, how='left',on='id')
deputados_join.head()

,id,nome,siglaPartido,ufLegislacao,email,escolaridade,dataNascimento,sexo,estadoNascimento,cidadeNascimento
0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR,SUPERIOR INCOMPLETO,1965-02-13,M,BA,SALVADOR
1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR,SUPERIOR,1966-11-06,M,SP,SÃO PAULO
2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR,SUPERIOR,1983-09-28,M,AP,MACAPÁ
3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR,SUPERIOR,1981-02-02,M,BA,SALVADOR
4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR,DOUTORADO,1969-03-06,F,SP,SÃO PAULO


###### 2 - REQUISIÇÃO LISTA DE GASTOS

In [69]:
gastos = []
for id in deputados.id:
    url_despesa = "https://dadosabertos.camara.leg.br/api/v2/deputados/"
    url_despesa = url_despesa +str(id)+"/despesas?pagina=2&ano=2019&ano=2020&itens=999999999"
    resposta = requests.get(url_despesa)
    gasto = pd.DataFrame(resposta.json()['dados'])
    gasto['id'] = id
    gastos.append(gasto)
    
total_gastos = pd.concat(gastos)
lista_exclusao_gastos = ['numRessarcimento','dataDocumento','codDocumento', 'codTipoDocumento','numDocumento', 'urlDocumento','codLote', 'parcela']
i = 0

while i < len(lista_exclusao_gastos):
    del total_gastos[lista_exclusao_gastos[i]] 
    i += 1
    
print('Processo de consulta finalizado')

Processo de consulta finalizado


### TRATAMENTO DOS DADOS 
Modelagem dos dados da requisição 'Lista de Gastos'. <hr>

<strong>Informações detalhadas sobre a lista de gastos:<strong> <br> 

In [76]:
total_gastos.count()

ano                  49933
mes                  49933
tipoDespesa          49933
tipoDocumento        49933
valorDocumento       49933
nomeFornecedor       49933
cnpjCpfFornecedor    49933
valorLiquido         49933
valorGlosa           49933
id                   49933
dtype: int64

Observações:
- Aqui já conseguimos ver se existe diferença entre o count das colunas, sendo possível verificar em quais existem valores nulos. 
- Neste caso não houve a necessidade de tratamento para o tipo dos dados, pois, todos estão retornando de forma correta: <br> Textos em Object/String e Valores calculáveis em Float.

<strong>Verificação e correção dos dados nulos ou duplicados: <strong> <br>

In [25]:
print('LISTAGEM DE COLUNAS COM VALORES NULOS:')
print(total_gastos.isnull().sum())
print('TOTAL DE DADOS VS COLUNAS:')
print(total_gastos.shape)
print('TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:')
total_gastos.drop_duplicates()
print(total_gastos.shape)

LISTAGEM DE COLUNAS COM VALORES NULOS:
ano                  0
mes                  0
tipoDespesa          0
tipoDocumento        0
valorDocumento       0
nomeFornecedor       0
cnpjCpfFornecedor    0
valorLiquido         0
valorGlosa           0
id                   0
dtype: int64
TOTAL DE DADOS VS COLUNAS:
(50910, 10)
TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:
(50910, 10)


Observações:
- Neste caso não houve necessidade de ajustes pois não há nenhum dado nulo ou duplicado. <br> 
É possível identificar através da contagem de valores nulos = 0 e também na diferença na contagem dos dados vs colunas com ou sem duplicações.

<strong>Analisando a coluna Fornecedores:<strong> <br> 

In [26]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.nomeFornecedor.value_counts().head(15))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.nomeFornecedor = [str(fornecedor).upper().replace("S.A.", "S.A").replace("S/A","S.A").replace("LTDA-ME","LTDA") for fornecedor in total_gastos.nomeFornecedor]
print(total_gastos.nomeFornecedor.value_counts().head(15))

DADOS FORNECEDORES SEM ALTERAÇÃO:
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA          3190
Cia Aérea - GOL                                 1367
CELULAR FUNCIONAL                               1022
Cia Aérea - TAM                                  869
RAMAL                                            860
WMS COMERCIO DE ARTIGOS DE PAPELARIA LTDA-ME     848
AUTO POSTO AEROPORTO LTDA                        839
FRATELLI POSTO DE COMBUSTIVEIS LTDA              478
CEMIG DISTRIBUIÇÃO S.A.                          409
AMORETTO CAFES EXPRESSO LTDA                     393
COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA     314
BRASAL COMBUSTIVEIS LTDA                         311
Cia Aérea - AVIANCA                              299
CLARO S.A.                                       288
CORREIOS - SEDEX CONVENCIONAL                    281
Name: nomeFornecedor, dtype: int64
----------------------------------------------
DADOS FORNECEDORES AJUSTADOS:
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA          3190
CIA AÉ

Observações:
- Identifiquei que o dado não está padronizado, há alguns com escrita em caixa alta e outros não. Também contém variações de escrita para o mesmo fornecedor.

<strong>Analisando a coluna Tipos de Despesas:<strong> <br> 

In [27]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.tipoDespesa.value_counts().head(20))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.tipoDespesa = [str(despesas).upper() for despesas in total_gastos.tipoDespesa]
print(total_gastos.tipoDespesa.value_counts().head(20))

DADOS FORNECEDORES SEM ALTERAÇÃO:
COMBUSTÍVEIS E LUBRIFICANTES.                                20781
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR    18741
Emissão Bilhete Aéreo                                         2834
TELEFONIA                                                     2726
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          1900
SERVIÇOS POSTAIS                                               930
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                  782
SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                      655
PASSAGENS AÉREAS                                               581
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                  325
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                     319
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.         171
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.        86
LOCAÇÃO OU FRETAMENTO DE AERONAVES                              29
ASSINATURA DE PUBLICAÇÕES   

Observações:
- Identifiquei que nessa coluna o dado também não está padronizado contém alguns com escrita em caixa alta e outros não. 

<strong>Analisando a colunas ano e mês:<strong> <br> 

In [71]:
import numpy as np
total_gastos.ano = [str(ano_list).replace("2020.0", "2020").replace("2019.0", "2019") for ano_list in total_gastos.ano]
total_gastos.groupby('ano').describe()

mes                                               valorDocumento  \
        count      mean       std  min  25%  50%  75%   max          count   
ano                                                                          
2019  50600.0  5.995296  3.557183  1.0  3.0  6.0  9.0  12.0        50600.0   
2020    310.0  2.774194  1.117952  1.0  2.0  3.0  3.0   5.0          310.0   

                   ... valorGlosa                 id                 \
             mean  ...        75%       max    count           mean   
ano                ...                                                
2019  1027.571693  ...        0.0  86000.00  50600.0  167303.042747   
2020  2002.772871  ...        0.0   1655.17    310.0  165868.177419   

                                                                     
               std      min       25%       50%       75%       max  
ano                                                                  
2019  45680.023529  66179.0  141555.0  178956.0  204453.0  212749.0  
2020  53007.034340  73692.0  128760.0  178982.0  205535.0  212749.0  

[2 rows x 40 columns]

In [15]:
total_gastos.ano = [str(ano_list).replace("2020.0", "2020").replace("2019.0", "2019") for ano_list in total_gastos.ano]
total_gastos.mes = [str(mes_list).replace("1.0", "1").replace("2.0", "2").replace("3.0", "3").replace("4.0", "4").replace("4.0", "4") for mes_list in total_gastos.mes]

Observações:
- Nessa variável fiz um ajuste para 'string' e retirei as casas decimais. 

Para os demais dados da requisição despesas não houve necessidade de ajustes:
- valorDocumento
- cnpjCpfFornecedor
- valorLiquido
- valorGlosa
- id

<strong>Renomeando o nome das colunas:<strong> <br> 

In [11]:
deputados.columns = ['ID', 'NOME', 'PARTIDO','UF','E-MAIL']
deputados.head()

,ID,NOME,PARTIDO,UF,E-MAIL
0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [12]:
total_gastos.columns = ['ANO', 'MÊS', 'TIPO-DESPESA','TIPO-DOCUMENTO','VALOR-DOCUMENTO','NOME-FORNECEDOR','CNPJ-FORNECEDOR','VALOR-LÍQUIDO','VALOR-GLOSA','ID']
total_gastos.head()

,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,284.36,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,281.03,3.33,204554
3,2020,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,80.96,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,80.96,0.00,204554
4,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554


### CRIANDO UMA BASE COM OS DADOS AJUSTADOS
Criando um CSV que salva todos os dados já tratados para análises sem a necessidade de requisições na API.
<hr>

In [1]:
deputados.to_csv('db/deputados.csv',encoding='utf8' )
total_gastos.to_csv('db/total_gastos.csv',encoding='utf8' )

NameError: name 'deputados' is not defined

In [16]:
deputados = pd.read_sql("select * from deputados;", engine)
deputados.head()

,index,ID,NOME,PARTIDO,UF,E-MAIL
0,0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [17]:
total_gastos = pd.read_sql("select * from gastos;", engine)
total_gastos.head()

,index,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,284.36,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,281.03,3.33,204554
3,3,2020,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,80.96,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,80.96,0.00,204554
4,4,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554
